In [ ]:
import BioSimSpace as BSS
import sys

In [ ]:
# Helper function
def runProcess(system, protocol, engine="AMBER", pmemd=False):
    """
    Given a solvated system (BSS object) and BSS protocol, run a process workflow with either
    Sander (CPU) or pmemd.cuda (GPU). NPT is typically done with GPU to save computing time.
    Returns the processed system.
    """

    # Create the process passing a working directory.
    if engine == "AMBER":
        if not pmemd:
            process = BSS.Process.Amber(system, protocol)
        elif pmemd:
            process = BSS.Process.Amber(system, protocol, exe=pmemd_path)
    elif engine == "GROMACS":
        process = BSS.Process.Gromacs(system, protocol)
    elif engine == "OpenMM":
        process = BSS.Process.OpenMM(system, protocol)
    # Start the process.
    process.start()

    # Wait for the process to exit.
    process.wait()

    # Check for errors.
    if process.isError():
        print(process.stdout())
        print(process.stderr())
        raise _Exceptions.ThirdPartyError("The process exited with an error!")

    # If it worked, try to get the system. No need to block, since it's already finished.
    system = process.getSystem()

    return system

In [ ]:
# Initialise the node object
node = BSS.Gateway.Node("Equilibrate a solvated ligand.")
# Set the node author and license.
node.addAuthor(
    name="Julien Michel",
    email="julien.michel@ed.ac.uk",
    affiliation="University of Edinburgh",
)
node.setLicense("GPLv3")

### Set the node inputs
node.addInput(
    "system",
    BSS.Gateway.FileSet(
        help="A topology/coordinate representation of a solvated complex."
    ),
)

node.addInput(
    "minsteps",
    BSS.Gateway.Integer(
        help="The max number of energy minimisation steps.", default=250
    ),
)

node.addInput(
    "nvt_restrained",
    BSS.Gateway.Time(
        help="The duration of the NVT restrained equilibration stage.",
        default=5,
        unit="picoseconds",
    ),
)
node.addInput(
    "nvt",
    BSS.Gateway.Time(
        help="The duration of the NVT restrained equilibration stage.",
        default=50,
        unit="picoseconds",
    ),
)
node.addInput(
    "npt",
    BSS.Gateway.Time(
        help="The duration of the NVT restrained equilibration stage.",
        default=200,
        unit="picoseconds",
    ),
)
node.addInput(
    "MDengine",
    BSS.Gateway.String(
        help="The MD engine to use for equilibration/minimisation.",
        allowed=["AMBER", "GROMACS", "OpenMM"],
        default="GROMACS",
    ),
)
### Set the node outputs
node.addOutput("system_eq", BSS.Gateway.FileSet(help="The equilibrated system."))

In [ ]:
node.showControls()

In [ ]:
#######################################
### Load the system  ##
#######################################
system = BSS.IO.readMolecules(node.getInput("system"))

In [ ]:
engine = node.getInput("MDengine")
print("The simulation engine is %s" % engine)

In [ ]:
# Minimisation #
print(f"Minimisation stage..")
protocol = BSS.Protocol.Minimisation(steps=node.getInput("minsteps"))
minimised = runProcess(system, protocol, engine=engine)

In [ ]:
print(f"NVT equilibration while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
    runtime=node.getInput("nvt_restrained"),
    temperature_start=0 * BSS.Units.Temperature.kelvin,
    temperature_end=300 * BSS.Units.Temperature.kelvin,
    restraint="all",
)
equil1 = runProcess(minimised, protocol, engine=engine)

In [ ]:
print(f"NVT equilibration while restraining all backbone atoms..")
protocol = BSS.Protocol.Equilibration(
    runtime=node.getInput("nvt"),
    temperature=300 * BSS.Units.Temperature.kelvin,
    restraint="backbone",
)
equil2 = runProcess(equil1, protocol, engine=engine)

In [ ]:
print(f"NVT equilibration  without restraints..")
protocol = BSS.Protocol.Equilibration(
    runtime=node.getInput("nvt"), temperature=300 * BSS.Units.Temperature.kelvin
)

equil3 = runProcess(equil2, protocol, engine=engine)

In [ ]:
print(f"NPT equilibration while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
    runtime=node.getInput("npt"),
    pressure=1 * BSS.Units.Pressure.atm,
    temperature=300 * BSS.Units.Temperature.kelvin,
    restraint="heavy",
)
equil4 = runProcess(equil3, protocol, engine=engine)

In [ ]:
print(f"NPT equilibration  without restraints..")
protocol = BSS.Protocol.Equilibration(
    runtime=node.getInput("npt"),
    pressure=1 * BSS.Units.Pressure.atm,
    temperature=300 * BSS.Units.Temperature.kelvin,
)
system_eq = runProcess(equil4, protocol, engine=engine)

In [ ]:
# Save systems
node.setOutput(
    "system_eq", BSS.IO.saveMolecules("bound_eq", system_eq, ["PRM7", "RST7"])
)

In [ ]:
node.validate()